In [28]:
import sys, time
import numpy as np
import arkouda as ak

In [29]:
ak.connect(server="localhost", port=5555)

In [48]:
# files and locations on my(mike's) laptop
hdffiles = ["/Users/valton/lanl_netflow/hdf5/netflow_day-02-00.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-01.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-02.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-03.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-04.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-05.hdf"]

In [49]:
fields = ['srcIP', 'dstIP', 'srcPort', 'dstPort', 'start']

In [50]:
%time nfDF = {field: ak.read_hdf(field, hdffiles) for field in fields}

CPU times: user 2 ms, sys: 2.1 ms, total: 4.11 ms
Wall time: 6.45 s


In [51]:
print(nfDF['start'])
print(nfDF['srcIP'])
print(nfDF['dstIP'])
print(nfDF['srcPort'])
print(nfDF['dstPort'])

[118781 118783 118785 ... 140369 140369 140369]
[34111590301 34111676798 48764590996 ... 34112191431 34112191431 34112191431]
[34111773101 34112055194 34111367844 ... 34111545319 31712083968 31712083968]
[5507 3137 5060 ... 78620 44270 49614]
[46272 445 5060 ... 95765 80 80]


In [52]:
print(nfDF)

{'srcIP': array([34111590301, 34111676798, 48764590996, ..., 34112191431, 34112191431, 34112191431]), 'dstIP': array([34111773101, 34112055194, 34111367844, ..., 34111545319, 31712083968, 31712083968]), 'srcPort': array([5507, 3137, 5060, ..., 78620, 44270, 49614]), 'dstPort': array([46272, 445, 5060, ..., 95765, 80, 80]), 'start': array([118781, 118783, 118785, ..., 140369, 140369, 140369])}


In [53]:
print(ak.info(ak.AllSymbols))

name:"id_43" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_42" dtype:"int64" size:100 ndim:1 shape:(100) itemsize:8
name:"id_47" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_46" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_45" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_44" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_30" dtype:"int64" size:80000000 ndim:1 shape:(80000000) itemsize:8
name:"id_37" dtype:"int64" size:60169 ndim:1 shape:(60169) itemsize:8
name:"id_38" dtype:"int64" size:60169 ndim:1 shape:(60169) itemsize:8



In [54]:
%time u,c = nfDF['srcIP'].unique(return_counts=True)

CPU times: user 1.47 ms, sys: 1.67 ms, total: 3.14 ms
Wall time: 13.1 s


In [55]:
print(u.size,u)
print(c.size,c)

25559 [34112110600 34111400624 48764978487 ... 48764444540 34111644622 34111308561]
25559 [3976 3960 176 ... 183 561 2159]


In [56]:
%time u,c = nfDF['dstIP'].unique(return_counts=True)

CPU times: user 1.42 ms, sys: 1.85 ms, total: 3.27 ms
Wall time: 12.2 s


In [57]:
print(u.size,u)
print(c.size,c)

23790 [34111337455 34111618066 34111947526 ... 34112119691 48764677226 48764683812]
23790 [39 12 72 ... 56 2 4]


In [58]:
%time u,c = nfDF['srcPort'].unique(return_counts=True)

CPU times: user 1.22 ms, sys: 1.64 ms, total: 2.85 ms
Wall time: 1.37 s


In [59]:
print(u.size,u)
print(c.size,c)

64899 [2 4 5 ... 99996 99997 99998]
64899 [188 223 2518 ... 284 220 220]


In [60]:
%time u,c = nfDF['dstPort'].unique(return_counts=True)

CPU times: user 1.06 ms, sys: 1.63 ms, total: 2.69 ms
Wall time: 1.31 s


In [61]:
print(u.size,u)
print(c.size,c)

59664 [2 4 7 ... 99996 99997 99998]
59664 [4 11 4698 ... 7 5 5]


In [62]:
print((nfDF['srcPort'] == 8080).sum())
print((nfDF['dstPort'] == 8080).sum())

2508
418986


In [63]:
s = u # pick off some dest ports
print(s)
%time t = ak.in1d(nfDF['dstPort'], s) # bool vector of where they are found in the data
print(t.sum())

[2 4 7 ... 99996 99997 99998]
CPU times: user 1.48 ms, sys: 5.1 ms, total: 6.58 ms
Wall time: 9.98 s
60000000


In [64]:
s = u.to_ndarray() # pick off some dest ports
dp = nfDF['dstPort'].to_ndarray()
print(s)
%time t = np.in1d(dp, s) # bool vector of where they are found in the data
print(t.sum())

[    2     4     7 ... 99996 99997 99998]
CPU times: user 7.47 s, sys: 1.04 s, total: 8.51 s
Wall time: 8.6 s
60000000


In [65]:
a = nfDF['start']
aMin = a.min()
aMax = a.max()
print(aMin,aMax, aMax - aMin)
%time h = ak.histogram(a,bins=100)
print(h)
print(h.sum(), a.size)

118781 140369 21588
CPU times: user 1.22 ms, sys: 417 µs, total: 1.64 ms
Wall time: 136 ms
[87 260 221 ... 507133 811672 769215]
60000000 60000000


In [66]:
#ak.shutdown()
#ak.disconnect()